In [1]:
import numpy as np
from scipy.stats import uniform, norm
import pandas as pd


def input_data_from_code():
    N = 100
    a = [0.05, 0.45, 0.09, -0.38]
    b = [0.6, 0.4, 0]
    p = 3
    q = 3
    return N, a, b, p, q


def generating_epsilon(N, error=1e-3):
    epsilon = np.array([])
    epsilon = uniform(loc=-error, scale=2 * error).rvs(size=N)
    return epsilon


def generating_v(N):
    v = np.array([])
    v = norm(loc=0, scale=1).rvs(size=N)
    return v


def generating_y(v, epsilon, N, a, b, p, q):
    y = np.array([])

    for m in range(0, max(p, q)):
        y = np.append(y, v[m])

    for i in range(max(p, q), N):
        summ = a[0]
        for autoregressive in range(1, p + 1):
            summ += a[autoregressive] * y[i - autoregressive]
        for moving_average in range(0, q):
            summ += b[moving_average] * v[i - moving_average - 1]
        summ += v[i]
        summ += epsilon[i]
        y = np.append(y, summ)
    return y


def create_matrix_x(y, v, N, p, q):
    x = np.array([[1] * (N - max(p, q))])
    for i in range(1, p + 1):
        x_column = np.array([])
        for k in range(max(p, q), N):
            x_column = np.append(x_column, y[k - i])
        x = np.append(x, [x_column], axis=0)
    for j in range(0, q + 1):
        x_column = np.array([])
        for k in range(max(p, q), N):
            x_column = np.append(x_column, v[k - j])
        x = np.append(x, [x_column], axis=0)

    return x.transpose()


def create_teta(a, b, p, q):
    teta = np.array([])
    for i in range(0, p + 1):
        teta = np.append(teta, a[i])
    teta = np.append(teta, 1.)
    for i in range(0, q):
        teta = np.append(teta, b[i])

    return teta


def resize_y(y, p):
    return y[p:]


def mnk(x, y):
    x_t = x.transpose()
    res = np.linalg.inv(x_t @ x) @ x_t @ y
    return res


def rmnk(x, y):
    teta_0 = np.array([[0] * (len(x[0]))]).transpose()
    beta = 10
    p_0 = np.eye(len(x[0])) * beta
    for i in range(len(x)):
        x_m = x[i].reshape(1, -1)
        x_m_t = x[i].reshape(-1, 1)
        p_1 = p_0 - (p_0.dot(x_m_t.dot(x_m.dot(p_0)))) / (1 + x_m.dot(p_0.dot(x_m_t)))
        teta_1 = teta_0 + p_1.dot(x_m_t) * (y[i] - x_m.dot(teta_0))
        teta_0 = teta_1
        p_0 = p_1
    return teta_0


def summ_squares_errors(y, y_1):
    error = np.linalg.norm(y_1 - y) ** 2
    return error


def r_2(y, y_1):
    return np.var(y_1) / np.var(y)


def calculation_of_the_Akaike_criterion(s, p, q, N):
    n = p + q + 1
    return N * np.log(s) + 2 * n

if __name__ == '__main__':
    N, a, b, p, q = input_data_from_code()

    epsilon = generating_epsilon(N)
    v = generating_v(N)

    s_mnk = np.array([])
    s_rmnk = np.array([])
    r_2_mnk = np.array([])
    r_2_rmnk = np.array([])
    ika_mnk = np.array([])
    ika_rmnk = np.array([])
    for i in range(3):
        for j in range(3):
            p = i + 1
            q = j + 1
            print("АРКС(" + str(p) + ", " + str(q) + ")")
            y = generating_y(v, epsilon, N, a, b, p, q)
            x = create_matrix_x(y, v, N, p, q)
            res_mnk = mnk(x, resize_y(y, max(p, q)))
            res_rmnk = rmnk(x, resize_y(y, max(p, q)))
            
            y_s_krushkoy_mnk = x @ res_mnk
            y_s_krushkoy_rmnk = x @ res_rmnk
            s_mnk = np.append(s_mnk, summ_squares_errors(resize_y(y, max(p, q)), y_s_krushkoy_mnk))
            s_rmnk = np.append(s_rmnk, summ_squares_errors(resize_y(y, max(p, q)), y_s_krushkoy_rmnk.transpose()))
            r_2_mnk = np.append(r_2_mnk, r_2(resize_y(y, max(p, q)), y_s_krushkoy_mnk))
            r_2_rmnk = np.append(r_2_rmnk, r_2(resize_y(y, max(p, q)), y_s_krushkoy_rmnk.transpose()))
            ika_mnk = np.append(ika_mnk,
                                calculation_of_the_Akaike_criterion(
                                    summ_squares_errors(resize_y(y, max(p, q)), y_s_krushkoy_mnk), p, q, N - max(p, q)))
            ika_rmnk = np.append(ika_rmnk,
                                 calculation_of_the_Akaike_criterion(
                                     summ_squares_errors(resize_y(y, max(p, q)), y_s_krushkoy_rmnk.transpose()), p, q, N - max(p, q)))


АРКС(1, 1)
АРКС(1, 2)
АРКС(1, 3)
АРКС(2, 1)
АРКС(2, 2)
АРКС(2, 3)
АРКС(3, 1)
АРКС(3, 2)
АРКС(3, 3)


In [2]:
print(res_rmnk)

[[ 0.04975447]
 [ 0.45755254]
 [ 0.08402341]
 [-0.37593836]
 [ 0.99871295]
 [ 0.59213587]
 [ 0.39769593]
 [-0.00480374]]
